## Fairness Results - new from 31.03.2025

In [ ]:
import pandas as pd
import numpy as np
import os

import metrics_normalized
import radar_chart
import fairness_dashboard

import pickle
import seaborn as sns
from scipy import stats
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#sensitive_attribute = "nationality"
sensitive_attribute = "gender"

#model = "baseline"
#model = "nosen"
#model = "fgbm"
#model = "hpt"
model="to"
#model = "roc"
#model = "eodds"

input_path = "xxx" # add customized local path to input folder
output_path = "xxx" # add customized local path to output folder

y_true = pd.read_pickle(input_path+"y_test.pkl")
y_pred = pd.read_pickle(input_path+"y_pred_te_baseline.pkl")
S_gen = pd.read_pickle(input_path+"S_gender_test.pkl")
S_nat = pd.read_pickle(input_path+"S_nat_test.pkl")
#y_pred_binary = (y_pred > 0.5).astype(int)

if model == "nosen":
    y_pred = pd.read_pickle(input_path+"y_pred_lgbm_nosen.pkl")[0].to_numpy()
if model == "fgbm":
    if sensitive_attribute == "gender":
        y_pred = pd.read_pickle(input_path+"y_pred_fgbm_gender.pkl")[0].to_numpy()
    else: y_pred = pd.read_pickle(input_path+"y_pred_fgbm_nat.pkl")[0].to_numpy()
if model == "hpt":
    if sensitive_attribute == "gender":
        y_pred = pd.read_pickle(input_path+"y_pred_fgbm_gen_hpt_v2.pkl")[0].to_numpy()
    else: y_pred = pd.read_pickle(input_path+"y_pred_nat_FGBM_HPT_model.pkl")[0].to_numpy()
if model == "to":
    y_pred = pd.read_pickle(input_path+"ypred_to_nat_tprr_recall.pkl")[0].to_numpy()
if model == "roc":
    if sensitive_attribute == "gender":
        y_pred = pd.read_csv(input_path+"y_pred_roc_gen_MF.csv")["0"].to_numpy().flatten()
        #y_pred = pd.read_csv(input_path+"y_pred_roc_gen_MU.csv")["0"].to_numpy().flatten()
        #y_pred = pd.read_csv(input_path+"y_pred_roc_gen_FU.csv")["0"].to_numpy().flatten()
    else: y_pred  = pd.read_csv(input_path+"y_pred_roc_nat_AU.csv")["0"].to_numpy().flatten()
        #y_pred = pd.read_csv(input_path+"y_pred_roc_nat_AU.pkl").to_numpy().flatten()
        #y_pred = pd.read_csv(input_path+"y_pred_roc_nat_NAU.pkl").to_numpy().flatten()
if model == "eodds":
    if sensitive_attribute == "gender":
        y_pred = pd.read_csv(input_path+"y_pred_eodds_gen_MF.csv").to_numpy().flatten()
        #y_pred = pd.read_csv(input_path+"y_pred_eodds_gen_MU.csv").to_numpy().flatten()
        #y_pred = pd.read_csv(input_path+"y_pred_eodds_gen_FU.csv").to_numpy().flatten()
    else: y_pred = pd.read_csv(input_path+"y_pred_eodds_nat_AU.csv").to_numpy().flatten()

In [ ]:
if sensitive_attribute == "gender":
    mapping  = {0: 'Male', 1: 'Female', 2: 'Unknown'}
    groups = np.array([mapping[val] for val in S_gen])
else:
    mapping  = {0: 'Austrian', 1: 'Non-Austrian', 2: 'Unknown'}
    groups = np.array([mapping[val] for val in S_nat])
    
df = pd.DataFrame({
    'Group': groups,
    'Pred': y_pred,
    'True':y_true
})
df.head(3)

In [ ]:
metrics_dict = {}

for group in df['Group'].unique():
    group_data = df[df['Group'] == group]
    y_true = group_data['True']
    y_pred = group_data['Pred']
    #metrics_dict[group] = metrics.group_stats(y_true, y_pred)
    metrics_dict[group] = metrics_normalized.group_stats(y_true, y_pred)

In [ ]:
df_metrics = pd.DataFrame.from_dict(metrics_dict,orient='index')
df_metrics.round(4)

In [ ]:
results = metrics_normalized.calculate_fairness_metrics(metrics_dict["Male"], metrics_dict["Female"])

In [ ]:
with open(output_path+'results_roc_MF.pkl', 'wb') as file:
    pickle.dump(results, file)

In [ ]:
results[1].popitem()
results[2].popitem()
radar_chart.plot_fairness_radar(results[1], results[2], filename=output_path+"radar_roc_MF.png")

In [ ]:
fairness_dashboard.bar_chart(results[3],results[4],output_path+"fairness_roc_MF.png")